In [11]:
import seaborn as sns
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()
import datetime
import warnings
import time
from scipy import stats
from sklearn import linear_model
import itertools
import calendar
warnings.filterwarnings('ignore')

# Back End Script 

### We are writing a python script to generate a table of insights for a particular user. This script will exist in one function, we can define functions within the function, but we want to streamline as much as possible for maximum efficiency. 

#### Insights To Generate  
- steps by sleep 
- sleep by heartrate 
- 

 _id: UUID (primary key)

owner: UUID (reference to User)

source: STRING - Describes where this insight came from. For example, insights created manually from the admin dashboard will have their source set as admin.

category: STRING - indicates which category/vertical the insight belongs to

type: STRING - A short identifier unique for each type of insight. Example: An insight recommending the user to take x steps to improve sleep might be identified by a type such as "step-count:sleep:achieve-steps".

message: TEXT - A textual representation of the insight, to be used in places where we can't display rich media, such as notifications. Example: "Get 2500 steps in by noon for a more restful sleep".

data: JSON - a JSON blob with detailed info on the insight. We make this a JSON field so that it can be easily extended in the future with new types of insights, without requiring changes to the DB schema.

upvote: BOOLEAN - A boolean field that is set to true when the user marks this insight as "useful" or gives it a "thumbs up". Default to false.

downvote: BOOLEAN - A boolean field that is set to true when the user marks this insight as "not useful" or gives it a "thumbs down". Default to false.


For the initial version, the JSON structure of the data field will be relatively simple:

{
    "message": "Get 2500 steps in by noon for a more restful sleep",
    "metrics": ["step-count", "sleep"],
    "explanation": "Science shows that getting physical activity and sunlight in the morning can help to keep your circadian rhythm in optimal shape. Your circadian rhythm is..."
}

In [12]:
#This is what the table is going to look like. 
#There are nine categories. Above is a short description of each column. 
UserInsight = pd.DataFrame(columns=['_id','owner','source','category','type','message','data','upvote','downvote'])
UserInsight

,_id,owner,source,category,type,message,data,upvote,downvote


In [13]:
#Example of how to insert into the table. 

blob = {"message":"If you keep losing weight","metrics":'The correlated variables',"explanation":'Leave this'}
example = ['000','John','Apple Watch','HeartRate','health-weight:heart-rate:positive','If you keep losing weight, your heart rate will drop',blob,'N/A','N/A']

UserInsight.loc[len(UserInsight)] = example
UserInsight

,_id,owner,source,category,type,message,data,upvote,downvote
0,000,John,Apple Watch,HeartRate,health-weight:heart-rate:positive,"If you keep losing weight, your heart rate wil...","{'message': 'If you keep losing weight', 'metr...",N/A,N/A


### Function  

In [14]:

data_sample = pd.read_csv('20171016-210106-DataSample.csv',dtype={"value": float})
data_sample2 = pd.read_csv('20171016-210304-DataSample.csv',dtype={"value": float})
data_sample3 = pd.read_csv('20171016-210529-DataSample.csv',dtype={"value": float})
data_sample4 = pd.read_csv('../20171031-235959-DataSample.csv',dtype={"value": float})



data_sample = data_sample.append([data_sample2,data_sample3,data_sample4])
data_sample['startDate'] = pd.to_datetime(data_sample['startDate']) 
data_sample['owner'].replace('00000000-5854-8d6f-b8eb-cf14a0f795df','00000000-56ff-538b-2223-e1800b5e3ddb',inplace=True)
data_sample['startDate'] = pd.to_datetime(data_sample['startDate'])
data_sample['endDate'] = pd.to_datetime(data_sample['endDate'])
data_sample.index = data_sample['startDate']

to_tdelta = lambda row: row['endDate'] - row['startDate']
data_sample['duration'] = data_sample.apply(to_tdelta, axis=1)
data_sample['day_of_week'] = data_sample['startDate'].dt.dayofweek
data_sample.sort_index(inplace=True)
data_sample.head()

,_id,owner,source,sourceId,sourceName,type,startDate,endDate,value,content,originalData,createdAt,updatedAt,duration,day_of_week
startDate,,,,,,,,,,,,,,,
2012-01-19 07:49:00,36e3fbf8-53bd-4f17-ae4c-314f59aab97e,00000000-56ff-538b-2223-e1800b5e3ddb,fitbit,6011012:1,Fitbit,health-sleep,2012-01-19 07:49:00,2012-01-19 07:51:00,0.0,NaN,NaN,2017-10-11 18:05:32.162199-07,2017-10-12 15:09:13.597742-07,00:02:00,3
2012-01-19 07:49:00,36e3fbf8-53bd-4f17-ae4c-314f59aab97e,00000000-56ff-538b-2223-e1800b5e3ddb,fitbit,6011012:1,Fitbit,health-sleep,2012-01-19 07:49:00,2012-01-19 07:51:00,0.0,NaN,NaN,2017-10-11 18:05:32.162199-07,2017-10-12 15:09:13.597742-07,00:02:00,3
2012-01-19 07:49:00,dee37862-6869-488e-b301-c21bc69e6f1e,00000000-56ff-538b-2223-e1800b5e3ddb,fitbit,6011012:1,Fitbit,health-sleep,2012-01-19 07:49:00,2012-01-19 07:51:00,0.0,NaN,NaN,2017-10-11 11:07:13.318881-07,2017-10-11 17:32:01.100877-07,00:02:00,3
2012-01-19 07:49:00,36e3fbf8-53bd-4f17-ae4c-314f59aab97e,00000000-56ff-538b-2223-e1800b5e3ddb,fitbit,6011012:1,Fitbit,health-sleep,2012-01-19 07:49:00,2012-01-19 07:51:00,0.0,NaN,NaN,2017-10-11 18:05:32.162199-07,2017-10-12 15:09:13.597742-07,00:02:00,3
2012-01-19 07:49:00,dee37862-6869-488e-b301-c21bc69e6f1e,00000000-56ff-538b-2223-e1800b5e3ddb,fitbit,6011012:1,Fitbit,health-sleep,2012-01-19 07:49:00,2012-01-19 07:51:00,0.0,NaN,NaN,2017-10-11 11:07:13.318881-07,2017-10-11 17:32:01.100877-07,00:02:00,3


In [45]:
#We need to tweak our functions to create an array like example above ^ 

class UserInsightFunction():
    
    def __init__(self, user_id, data_table):
        self.UserInsight = pd.DataFrame(columns=['_id','owner','source','category','type','message','data','upvote','downvote'])
        self.InsightTypes = []
        self.user_id = user_id
        self.data_table = data_table
        
    def prepare_table(self, x, y,time='D',data_sample=data_sample):
        steps = data_sample[data_sample['type'] == x]
        bodyfat = data_sample[data_sample['type'] == y]


        steps.index = steps['startDate']
        bodyfat.index = bodyfat['startDate']


        steps['value'] = pd.to_numeric(steps['value'])
        steps = steps.loc[:,'value']
        if x == 'health-step-count':
            steps = steps.resample(time).sum()
        else:
            steps = steps.resample(time).mean()


        bodyfat['value'] = pd.to_numeric(bodyfat['value'])
        bodyfat = bodyfat.loc[:,'value']
        if y == 'health-step-count':
            bodyfat = bodyfat.resample(time).sum()
        else:
            bodyfat = bodyfat.resample(time).mean()

        steps = steps.to_frame('first')
        steps['second'] = bodyfat

        return steps

    def corr_generator(self, steps):
        steps = steps.dropna()
        x = steps['first'].values
        y = steps['second'].values
        x = x.reshape(len(x), 1)
        y = y.reshape(len(x), 1)
        if not (x.shape == (0, 1) or y.shape == (0, 1)):
            regr = linear_model.LinearRegression()
            regr.fit(x, y)
            corr_info = stats.stats.pearsonr(x, y)
            r = corr_info[0][0]
            if corr_info[1] == 0.0:
                p = corr_info[1]
            else:
                p = corr_info[1][0]
            return r, p
        else:
            return -1.1, -1.1
    
    def CorrelationTable(self): 
        #create table of all correlations in our data set for the user.
        CorrelationTable = pd.DataFrame(columns=['Variable1','Variable2','Overall','Last 30 Days','Last 60 Days','Last 90 Days', 'p-Overall','p-Last 30 Days','p-Last 60 Days','p-Last 90 Days'])
        data = self.data_table[self.data_table['owner'] == self.user_id]
        types = [types for types in data.type.unique()]
        data30 = data.last('30D')
        data7 = data.last('60D')
        data1 = data.last('90D')
        for type1, type2 in itertools.permutations(types, 2):
            # print(type1 + " " + type2)
            r, p = self.corr_generator(self.prepare_table(type1, type2, data_sample=data))
            #print(type1 + " " + type2 + " " + str(r) + " total")
            r30, p30 = self.corr_generator(self.prepare_table(type1, type2, data_sample=data30))
            #print(type1 + " " + type2 + " " + str(r30) + " 30")
            r7, p7 = self.corr_generator(self.prepare_table(type1, type2, data_sample=data7))
            #print(type1 + " " + type2 + " " + str(r7) + " 7")
            r1, p1 = self.corr_generator(self.prepare_table(type1, type2, data_sample=data1))
            #print(type1 + " " + type2 + " " + str(r1) + " 1")
            curRow = {'Variable1': type1, 'Variable2': type2, 'Overall': r, 'Last 30 Days': r30, 'Last 60 Days': r7, 'Last 90 Days': r1, 'p-Overall': p, 'p-Last 30 Days': p30, 'p-Last 60 Days': p7, 'p-Last 90 Days': p1}
            CorrelationTable = CorrelationTable.append(curRow, ignore_index=True)
            
        return CorrelationTable
        
    
    def InsightCreator(self):
        #This is where we want to create the insight arrays from the correlation table, and append to UserInsights
        return

In [46]:
userinsights = UserInsightFunction('00000000-56ff-538b-2223-e1800b5e3ddb', data_sample)
corTable = userinsights.CorrelationTable()

In [47]:
corTable

,Variable1,Variable2,Overall,Last 30 Days,Last 60 Days,Last 90 Days,p-Overall,p-Last 30 Days,p-Last 60 Days,p-Last 90 Days
0,health-sleep,health-bmi,-0.501918,-0.294848,-0.723807,-0.724778,3.13033e-08,2.204122e-01,1.341572e-05,1.844132e-06
1,health-sleep,health-weight,-0.490629,-0.037098,-0.597419,-0.648200,2.25787e-08,8.665436e-01,1.911512e-04,4.600162e-06
2,health-sleep,daily-summary:health-sleep,0.516831,-0.046289,0.206097,0.433346,1.51161e-09,8.463439e-01,3.124439e-01,8.286959e-03
3,health-sleep,health-height,-1.100000,-1.100000,-1.100000,-1.100000,-1.1,-1.100000e+00,-1.100000e+00,-1.100000e+00
4,health-sleep,health-fat-free-mass,-0.016976,0.082104,-0.216445,-0.187455,0.870297,7.802200e-01,3.212127e-01,3.394764e-01
5,health-sleep,health-body-fat,-0.143259,-0.194740,-0.439287,-0.539727,0.163787,5.046781e-01,3.597324e-02,3.033790e-03
6,health-sleep,health-fat-mass-weight,-0.620489,-0.219050,-0.595393,-0.656250,1.99179e-11,4.518076e-01,2.723750e-03,1.494076e-04
7,health-sleep,health-heart-rate,-0.059381,0.451372,0.204507,0.068060,0.492267,2.352245e-02,1.829857e-01,5.991579e-01
8,health-sleep,health-step-distance,-0.115453,0.206327,-0.012363,0.003093,0.18239,5.427350e-01,9.522011e-01,9.859329e-01
9,health-sleep,health-step-count,0.082985,-0.000114,-0.282567,-0.111385,0.264066,9.995702e-01,6.001246e-02,3.695249e-01


In [64]:
corTable['Variable1'].unique()

array(['health-sleep', 'health-bmi', 'health-weight',
       'daily-summary:health-sleep', 'health-height',
       'health-fat-free-mass', 'health-body-fat', 'health-fat-mass-weight',
       'health-heart-rate', 'health-step-distance', 'health-step-count',
       'daily-summary:health-step-count', 'daily-summary:health-heart-rate'], dtype=object)

In [65]:
variables = {'health-sleep': 'sleep', 'health-bmi': 'Body Mass Index', 'health-weight': 'weight',
       'daily-summary:health-sleep': 'sleep', 'health-height': 'height',
       'health-fat-free-mass': 'fat-free mass', 'health-body-fat': 'body fat', 'health-fat-mass-weight': 'fat-mass-weight',
       'health-heart-rate': 'heart rate', 'health-step-distance': 'step distance', 'health-step-count': 'step count',
       'daily-summary:health-step-count': 'step-count', 'daily-summary:health-heart-rate': 'heart rate'}

In [220]:
def create_insights(c_table, owner):
    
    def insight_from_corr(row): 
        
        out = pd.DataFrame()
        source = 'ongo'
        category = row.Variable2
        upvote, downvote = False, False
        for time_range in ['Overall', 'Last 30 Days', 'Last 60 Days', 'Last 90 Days']:
            val = row[time_range]
            p_val = row['p-' + time_range]
            if abs(val) > 0.3 and p_val <= 0.1:
                direction = 'positive' if val > 0 else 'negative'
                type_ = str(row['Variable1']) +':'+ str(row['Variable2']) +':'+ direction +':'+ time_range
                if time_range == 'Overall':
                    if direction == 'positive':
                        message1 = 'Your overall ' + variables[row['Variable1']] + ' and your overall ' + variables[row['Variable2']] + ' are correlated positively.'
                        message2 = ' An increase in ' + variables[row['Variable1']] + ' will continue to increase your ' + variables[row['Variable2']] + '.'
                    if direction == 'negative':
                        message1 = 'Your overall ' + variables[row['Variable1']] + ' and your overall ' + variables[row['Variable2']] + ' are correlated negatively.'
                        message2 = ' An increase in ' + variables[row['Variable1']] + ' will continue to decrease your ' + variables[row['Variable2']] + '.'
                else:
                    if direction == 'positive':
                        message1 = 'Your ' + variables[row['Variable1']] + ' in the ' + time_range + ' and your ' + variables[row['Variable2']] + ' in the ' + time_range + ' are correlated positively.'
                        message2 = ' An increase in ' + variables[row['Variable1']] + ' will continue to increase your ' + variables[row['Variable2']] + '.'
                    if direction == 'negative':
                        message1 = 'Your ' + variables[row['Variable1']] + ' in the ' + time_range + ' and your ' + variables[row['Variable2']] + ' in the ' + time_range + ' are correlated negatively.'
                        message2 = ' An increase in ' + variables[row['Variable1']] + ' will continue to decrease your ' + variables[row['Variable2']] + '.'
                message = message1 + message2
                insight = {
                    'owner': owner,
                    'source': source,
                    'category': category,
                    'type': type_,
                    'message': message
                }
                data = insight.copy()
                data.update({
                    'R': val,
                    'p': p_val
                })
                insight.update({
                    'upvote': upvote,
                    'downvote': downvote,
                    'data': data
                })
                out = out.append(pd.DataFrame(insight, index = [len(out)]))
            else:
                insight_ = {
                    'owner': 'N/A',
                    'source': 'N/A',
                    'category': 'N/A',
                    'type': 'N/A',
                    'message': 'N/A',
                    'upvote': 'N/A',
                    'downvote': 'N/A',
                    'data': 'N/A'
                }
                out = out.append(pd.DataFrame(insight_, index = [len(out)]))
        return out
    
    df = pd.DataFrame()
    for index, row in c_table.iterrows():
        df = df.append(insight_from_corr(row), ignore_index = True)
    df._id = df.index
    return df

## Final Product

In [227]:
def insights(owner):
    return create_insights(UserInsightFunction(owner, data_sample).CorrelationTable(), owner)

In [228]:
insights('00000000-56ff-538b-2223-e1800b5e3ddb')

,category,data,downvote,message,owner,source,type,upvote
0,health-bmi,NaN,False,Your overall sleep and your overall Body Mass ...,00000000-56ff-538b-2223-e1800b5e3ddb,ongo,health-sleep:health-bmi:negative:Overall,False
1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,health-bmi,NaN,False,Your sleep in the Last 60 Days and your Body M...,00000000-56ff-538b-2223-e1800b5e3ddb,ongo,health-sleep:health-bmi:negative:Last 60 Days,False
3,health-bmi,NaN,False,Your sleep in the Last 90 Days and your Body M...,00000000-56ff-538b-2223-e1800b5e3ddb,ongo,health-sleep:health-bmi:negative:Last 90 Days,False
4,health-weight,NaN,False,Your overall sleep and your overall weight are...,00000000-56ff-538b-2223-e1800b5e3ddb,ongo,health-sleep:health-weight:negative:Overall,False
5,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
6,health-weight,NaN,False,Your sleep in the Last 60 Days and your weight...,00000000-56ff-538b-2223-e1800b5e3ddb,ongo,health-sleep:health-weight:negative:Last 60 Days,False
7,health-weight,NaN,False,Your sleep in the Last 90 Days and your weight...,00000000-56ff-538b-2223-e1800b5e3ddb,ongo,health-sleep:health-weight:negative:Last 90 Days,False
8,daily-summary:health-sleep,NaN,False,Your overall sleep and your overall sleep are ...,00000000-56ff-538b-2223-e1800b5e3ddb,ongo,health-sleep:daily-summary:health-sleep:positi...,False
9,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
